<a href="https://colab.research.google.com/github/NanoShark/Navigating-Neurological-Realms/blob/main/CalculatesMeanIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
#calcualte the thresholds


import pandas as pd
import numpy as np
import glob

def calculate_ia(L, R):
    """Calculate Index of Asymmetry (IA)"""
    return abs((L - R) / max(L, R)) * 100

def analyze_asymmetry(data):
    # Extract relevant columns for left and right controllers
    left_position_cols = ['LeftPosX', 'LeftPosY', 'LeftPosZ']
    right_position_cols = ['RightPosX', 'RightPosY', 'RightPosZ']
    left_rotation_cols = ['LeftRotX', 'LeftRotY', 'LeftRotZ', 'LeftRotW']
    right_rotation_cols = ['RightRotX', 'RightRotY', 'RightRotZ', 'RightRotW']
    left_velocity_cols = ['LeftVelX', 'LeftVelY', 'LeftVelZ']
    right_velocity_cols = ['RightVelX', 'RightVelY', 'RightVelZ']

    # Initialize lists to store IA values for each type of measurement
    ia_positions = []
    ia_rotations = []
    ia_velocities = []

    # Calculate IA for positions
    for left_col, right_col in zip(left_position_cols, right_position_cols):
        for index, row in data.iterrows():
            ia_positions.append(calculate_ia(row[left_col], row[right_col]))

    # Calculate IA for rotations
    for left_col, right_col in zip(left_rotation_cols, right_rotation_cols):
        for index, row in data.iterrows():
            ia_rotations.append(calculate_ia(row[left_col], row[right_col]))

    # Calculate IA for velocities
    for left_col, right_col in zip(left_velocity_cols, right_velocity_cols):
        for index, row in data.iterrows():
            ia_velocities.append(calculate_ia(row[left_col], row[right_col]))

    # Calculate mean IA for positions, rotations, and velocities
    mean_ia_positions = np.mean(ia_positions)
    mean_ia_rotations = np.mean(ia_rotations)
    mean_ia_velocities = np.mean(ia_velocities)

    # Combine results into a dictionary
    asymmetry_results = {
        'mean_ia_positions': mean_ia_positions,
        'mean_ia_rotations': mean_ia_rotations,
        'mean_ia_velocities': mean_ia_velocities
    }

    return asymmetry_results

def determine_thresholds_for_healthy(alpha=1.2):
    """Determine the thresholds based on healthy individuals' data"""
    position_ias = []
    rotation_ias = []
    velocity_ias = []

    # Process all CSV files in the directory
    for file in glob.glob("path_to_healthy_csv_files/*.csv"):  # Replace with the correct path
        data = pd.read_csv(file)
        asymmetry_results = analyze_asymmetry(data)

        # Collect IA values for each measurement type
        position_ias.append(asymmetry_results['mean_ia_positions'])
        rotation_ias.append(asymmetry_results['mean_ia_rotations'])
        velocity_ias.append(asymmetry_results['mean_ia_velocities'])

    # Calculate mean IA across all healthy individuals
    mean_position_ia = np.mean(position_ias)
    mean_rotation_ia = np.mean(rotation_ias)
    mean_velocity_ia = np.mean(velocity_ias)

    # Determine thresholds
    thresholds = {
        'position_threshold': alpha * mean_position_ia,
        'rotation_threshold': alpha * mean_rotation_ia,
        'velocity_threshold': alpha * mean_velocity_ia
    }

    return thresholds

def main():
    # Determine thresholds using healthy individuals' data
    thresholds = determine_thresholds_for_healthy(alpha=1.2)

    # Output the determined thresholds
    print("Determined Thresholds for Asymmetry:")
    print(thresholds)

if __name__ == "__main__":
    main()
